In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Installing the necessary files
!pip install wikipedia
!pip install rake-nltk
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
#importing other necessary libraries
import wikipedia as wki
from textblob import TextBlob as txb
import pandas as pd
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math
import statistics
from operator import itemgetter
import random
import gensim
from gensim import corpora,models,similarities
from rake_nltk import Metric, Rake
import spacy
import en_core_web_lg
import collections
from collections import OrderedDict
import operator
nlp = en_core_web_lg.load()

In [0]:
# Get the WIKI dump of FEVER
table = []
nums = [f"{i:03}" for i in range(1,110)]
for i in nums:
    with open('/content/drive/My Drive/NLP/wiki-pages/wiki-'+i+'.jsonl') as f:
        for line in f:
          table.append(json.loads(line))

In [0]:
#Get the test Claims that needs to be labelled
test_claims = []
with open('/content/drive/My Drive/NLP/shared_task_dev_public.jsonl', 'r') as f:
    for line in f:
        test_claims.append(json.loads(line))

In [6]:
# This libraries will help to deploy model to web app
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
# from flask_debug import Debug
import pickle

In [7]:
!pip install transformers
!pip install jsonlines

In [0]:
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

In [0]:
# Load pre-trained model and tokenizer 
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
#Code for running the Web app
import numpy as np
import jsonlines
import re
# app = Flask(__name__)
app = Flask(__name__, root_path='/content/drive/My Drive/')
run_with_ngrok(app)
def remove_string_special_characters(s):
    stripped =  re.sub('[^\w\s]', '', s)
    stripped = re.sub('_', '', stripped)
    stripped = re.sub('\s+', ' ', stripped)
    stripped = stripped.strip()
    return stripped

def segregateSourceSentId(sorted_dict,temp2):
  newList=[]
  linesList=[]
  eachsource=[]
  if sorted_dict:
    for keyString in sorted_dict.keys():
      temp=[]
      source,id=keyString.split("\t") 
      eachsource.append(str(source))
      temp.append(str(source))
      temp.append(id)
      newList.append(temp)
      linesList.append(str(temp2.get(keyString))+"\n")       
  return eachsource,newList,linesList

def similarityCheck(claim,eachLine):
  token1=nlp(claim)
  token2=nlp(eachLine)
  return token1.similarity(token2)

def splitByEachLine(result,claim,lines):
  sourceLineIdScore={}
  sourceLineIdsentence={}
  allLine=lines.split('\n')
  i=0
  for eachLine in allLine:
    cleanSentence=remove_string_special_characters(eachLine)
    score=similarityCheck(claim,eachLine)
    key=(str(result)+"\t"+str(i))
    sourceLineIdScore[key]=score
    sourceLineIdsentence[key]=eachLine
    i+=1
  #print(sentenceWithScore)
  return sourceLineIdScore,sourceLineIdsentence

#define example and feature classes
class InputExample(object):
  def __init__(self, claim, ev, label=None):
    self.claim = claim
    self.ev = ev
    self.label = label

class InputFeatures(object):
    def __init__(self, claim_ids, ev_ids, label_id):
        self.claim_ids = claim_ids
        self.ev_ids = ev_ids
        self.label_id = label_id

def data_to_examples(line):
  examples = []
  #data_file=json.dumps(data1)
  #data = jsonlines.open(data_file)
  #labels = jsonlines.open('train_output.jsonl')
  #for line,line2 in zip(data,labels):
  #for index in range(845):
  #line = data.read()
# label = line2['label']
  claim = line['claim']
  ev=[]
  if line['evidences'] != 'null':
    for item in line['evidences']:
      #remove nonsense from sents
      clean = re.sub('\n+',' ',item)
      clean = re.sub('\s+',' ',clean)
      clean = re.sub('[a-zA-Z0-9]*\t[a-zA-Z0-9]*','',clean)
      clean = re.sub('-...-','',clean)
      if len(clean.split()) <= 512:
        ev.append(clean)
  else:
    ev.append('null')
  examples.append(InputExample(claim,ev))#,label))
  return examples

def example_to_features(example):
  #tokenize and add [CLS]/[SEP] tokens
  claim_tokens = tokenizer.encode(example.claim, add_special_tokens=True)
  ev_tokens = []
  if example.ev ==[]:
    ev_tokens.append([101])
  for item in example.ev:
    ev_tokens.append(tokenizer.encode(item,add_special_tokens=True))
  
  return InputFeatures(claim_tokens,ev_tokens,example.label)


loaded_model = pickle.load(open('/content/drive/My Drive/NLP/new_trained_model_15k.sav', 'rb'))      


#print(type(lines))

@app.route('/')
def home():
  return render_template('index.html')

@app.route('/test',methods=['POST'])
def test():
  # claim = request.form["claim_text_area"]
  
  # claim = [request.form.values()]

  claim=request.form['claim']
  rake = Rake("/content/drive/My Drive/NLP/stopwords.txt")
  rake = Rake(min_length=2, max_length=5)
  result={}
  #counter += 1
  phrases=[]   
  #temp_str=claims['claim']
  #rake.extract_keywords_from_text(temp_str)
  rake.extract_keywords_from_text(claim)
  phrases=rake.get_ranked_phrases()
  phrases.append(claim)
  temp_results=[]
  sourceLineIdScoreResultWise=[]
  temp1={}
  temp2={}
  sourceLineIdsentenceResultWise=[]
  if(len(phrases) != 0):
    for each in phrases:
      temp_results.extend(wki.search(each,results=1))
    for result in temp_results:
      if result is not 'null':
        temp = result.replace(" ", "_")
        try:
          lines = str(next(item['lines'] for item in table if item["id"] == temp))
          text_sents = sent_tokenize(lines)
          sourceLineIdScore,sourceLineIdsentence=splitByEachLine(result,claim,lines)
          sourceLineIdScoreResultWise.append(sourceLineIdScore)
          sourceLineIdsentenceResultWise.append(sourceLineIdsentence)
        except StopIteration:
          continue
        
        for eachDict in sourceLineIdScoreResultWise:
          temp1.update(eachDict)
        for eachDict in sourceLineIdsentenceResultWise:
          temp2.update(eachDict)
  d_descending = sorted(temp1.items(), key=operator.itemgetter(1),reverse=True)[:3] #sort by values i.e top 3 values in descending order 
  sorted_dict = dict(OrderedDict(d_descending))
  eachSource,sourcess,evidencess=segregateSourceSentId(sorted_dict,temp2)
  output_row = {}
  output_row['claim'] = claim
  output_row['sources'] = sourcess
  output_row['evidences'] = evidencess
  # json.dump(output_row, outfile);
  # outfile.write("\n");
  # print(counter)
  #return output_row

  test_ex = data_to_examples(output_row)

  feat_list = []
  for example in test_ex:
    features = example_to_features(example)
    feat_list.append(features)


  #####BOTTLENECK STEP
#create BERT representations for each example
  max_len = 432
  BERT_examples = []
  round = 0
  for item in feat_list:
    #padding
    padded_claim = np.array(item.claim_ids + [0]*(max_len-len(item.claim_ids)))
    padded_claim = np.reshape(padded_claim,(1,max_len))
    padded_ev = np.array([sent + [0]*(max_len-len(sent)) for sent in item.ev_ids])
    
    #mask padded tokens
    claim_mask = np.where(padded_claim != 0,1,0)
    ev_mask = np.where(padded_ev != 0,1,0)

    #create matrix: each row represents one sentence
    padded_total = np.concatenate((padded_claim,padded_ev),axis=0)
    mask_total = np.concatenate((claim_mask,ev_mask))

    #convert to tensors
    input_ids = torch.tensor(padded_total)
    attention_mask = torch.tensor(mask_total)

    #get BERT embeddings
    with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)

    #extract [CLS] embeddings and labels for input to classifier
    features = last_hidden_states[0][:,0,:].numpy()
    concatenated = features.flatten()
    BERT_examples.append(list(concatenated))
    round += 1
  
  #pad each example for uniform input size
  max_len = 109824

  padded_BERT = []
  for example in BERT_examples:
    padded_BERT.append(example + [0]*(max_len-len(example)))
  
  test_inputs = np.array([np.array(ex) for ex in padded_BERT])
  
  predictions = loaded_model.predict(test_inputs)
  answer="The claim \""+claim+"\""
  label_color="red"
  if predictions[0]=="SUPPORTS":
    result1="IS TRUE (SUPPORTS)"
  elif predictions[0]=="REFUTES":
    result1="IS FALSE (REFUTES)"
  else:
    result1="Cannot be determined"

  # if eachSource:
  #   for i in range(len(eachSource)):
  #     if(i==0):
  #       source_text0=eachSource[0]
  #     elif(i==1):
  #       source_text1=eachSource[1]
  #     elif(i==2):
  #       source_text2=eachSource[2]  

  #sourceEvidence = remove_string_special_characters(s)
  #tohit=np.array(claim)
  return render_template('index.html', result_text="{}".format(result1),claim_text='{}'.format(answer))
  # return render_template('index.html',source_text0="{}".format(source_text0),source_text1="{}".format(source_text1),source_text2="{}".format(source_text2), result_text="{}".format(result1),claim_text='{}'.format(answer))

if __name__ == "__main__":
  # Debug(app)
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://51c753c6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/May/2020 21:58:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2020 21:58:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2020 21:58:36] "POST /test HTTP/1.1" 200 -
